# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [134]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [201]:
vac_df = pd.read_csv("../../output_data/weather_data.csv")
vac_df.head(10)

,City,Lat,Lng,Max temp,Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,Butaritari,3.0707,172.7902,300.93,300.93,76,79,5.01,KI,1639606038
1,Andreyevka,55.9803,37.1350,271.93,271.77,86,40,5.00,RU,1639606038
2,Lompoc,34.6391,-120.4579,288.46,286.13,59,1,4.63,US,1639606038
3,Dikson,73.5069,80.5464,242.57,242.57,100,99,2.13,RU,1639606038
4,Cherskiy,68.7500,161.3000,241.19,241.19,80,80,1.99,RU,1639606038
5,Rikitea,-23.1203,-134.9692,298.36,298.36,72,83,3.88,PF,1639606039
6,Batagay,67.6333,134.6333,234.28,234.28,97,87,1.06,RU,1639606040
7,Punta Arenas,-53.1500,-70.9167,285.99,285.99,62,94,9.47,CL,1639605807
8,Coquimbo,-29.9533,-71.3436,290.87,290.87,73,3,2.93,CL,1639605802


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [281]:
# Store latitude and longitude in locations
vlocations = vac_df[["Lat","Lng"]]

In [282]:
# Store weight values and convert to float
hum = vac_df["Humidity"].astype(float)
hum

0     76.0
1     86.0
2     59.0
3    100.0
4     80.0
5     72.0
6     97.0
7     62.0
8     73.0
Name: Humidity, dtype: float64

In [283]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(vlocations, weights=hum, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=5)



# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

* Narrow down the DataFrame to find your ideal weather condition. For example:
* A max temperature lower than 80 degrees but higher than 70.
* Wind speed less than 10 mph.
* Zero cloudiness.
* Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
* **Note:** Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

In [270]:
Tindex = vac_df[(vac_df['Max temp']>=70) & (vac_df['Max temp']<=80) & (vac_df['Windspeed']<=10) & (vac_df['Cloudiness']<=0)].index
Tindex

Int64Index([], dtype='int64')

In [251]:
hotel_df = vac_df.drop(Tindex,inplace=False)
hotel_df

,City,Lat,Lng,Max temp,Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,Butaritari,3.0707,172.7902,300.93,300.93,76,79,5.01,KI,1639606038
3,Dikson,73.5069,80.5464,242.57,242.57,100,99,2.13,RU,1639606038
5,Rikitea,-23.1203,-134.9692,298.36,298.36,72,83,3.88,PF,1639606039
6,Batagay,67.6333,134.6333,234.28,234.28,97,87,1.06,RU,1639606040
7,Punta Arenas,-53.1500,-70.9167,285.99,285.99,62,94,9.47,CL,1639605807


# Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [252]:
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Lat,Lng,Max temp,Temp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name
0,Butaritari,3.0707,172.7902,300.93,300.93,76,79,5.01,KI,1639606038,
3,Dikson,73.5069,80.5464,242.57,242.57,100,99,2.13,RU,1639606038,
5,Rikitea,-23.1203,-134.9692,298.36,298.36,72,83,3.88,PF,1639606039,
6,Batagay,67.6333,134.6333,234.28,234.28,97,87,1.06,RU,1639606040,
7,Punta Arenas,-53.1500,-70.9167,285.99,285.99,62,94,9.47,CL,1639605807,


In [269]:
# params dictionary to update each iteration
list= []
params = {
    "radius": 5000,
    "types": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_hotel = requests.get(base_url, params=params)

    # convert to json
    name_hotel = name_hotel.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index,"Hotel Name"] = name_hotel["results"][index]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping: " + hotel_df["Hotel Name"])

In [258]:
hotel_df.head(10)

,City,Lat,Lng,Max temp,Temp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name
0,Butaritari,3.0707,172.7902,300.93,300.93,76,79,5.01,KI,1639606038,Ukiangang
3,Dikson,73.5069,80.5464,242.57,242.57,100,99,2.13,RU,1639606038,Tserkov' Nikolaya Chudotvortsa P. Dikson
5,Rikitea,-23.1203,-134.9692,298.36,298.36,72,83,3.88,PF,1639606039,Couvent de Rouru
6,Batagay,67.6333,134.6333,234.28,234.28,97,87,1.06,RU,1639606040,"Pamyatnik Mamonta, D'aany Uluuha, Verkhoyanski..."
7,Punta Arenas,-53.1500,-70.9167,285.99,285.99,62,94,9.47,CL,1639605807,Hotel Almasur Punta Arenas


In [266]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

# Add marker layer on top of heat map
# Display figure

In [267]:
# Convert hotel name to list
names = hotel_df["Hotel Name"].tolist()
names

['Ukiangang',
 "Tserkov' Nikolaya Chudotvortsa P. Dikson",
 'Couvent de Rouru',
 "Pamyatnik Mamonta, D'aany Uluuha, Verkhoyanskiy Rayon",
 'Hotel Almasur Punta Arenas']

In [268]:
# Create a map using state centroid coordinates to set markers
marker_locations = hotel_df[['Lat','Lng']]

# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Hotel: {name}" for name in names])
fig.add_layer(markers)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))